# Taylor appproximation

In [3]:
import logging

import numpy

import cicada
from cicada.additive import AdditiveProtocolSuite
from cicada.communicator import SocketCommunicator

from statistics import mean, stdev
from tqdm import *

logging.basicConfig(level=logging.INFO)

def main(communicator):
    log = cicada.Logger(logging.getLogger(), communicator)
    protocol = AdditiveProtocolSuite(communicator)

    values = numpy.array(1)
    log.info(f"Player {communicator.rank} values: {values}")

    values_share = protocol.share(src=0, secret=values, shape=())
    tay_tanh_share = protocol.taylor_approx(numpy.tanh, values_share)
    tay_tanh = protocol.reveal(tay_tanh_share)
    log.info(f"Player {communicator.rank} tay_tanh({values}): {tay_tanh} vs tanh({values}): {numpy.tanh(values)}")
    abs_err_list = []
    for i in tqdm(numpy.arange(-5, 5, .1)):
        cic_tay_tanh = protocol.reveal(protocol.taylor_approx(numpy.tanh, protocol.share(src=0, secret=numpy.array(i), shape=()), degree=10))
        legit_tanh = numpy.tanh(i)
        abs_err_list.append(abs(legit_tanh-cic_tay_tanh))

    
    if communicator.rank ==1:
        print(mean(abs_err_list), stdev(abs_err_list))

SocketCommunicator.run(world_size=3, fn=main);

INFO:root:Player 0 values: 1
INFO:root:Player 1 values: 1
INFO:root:Player 2 values: 1
INFO:root:Player 0 tay_tanh(1): 0.7615814208984375 vs tanh(1): 0.7615941559557649
INFO:root:Player 1 tay_tanh(1): 0.7615814208984375 vs tanh(1): 0.7615941559557649
INFO:root:Player 2 tay_tanh(1): 0.7615814208984375 vs tanh(1): 0.7615941559557649
100%|█████████████████████████████████████████████████████████████████████████████████| 100/100 [00:17<00:00,  5.77it/s]




192700.13701113572 402054.00249001564
